In [4]:
# install packages
! pip install pandas
! pip install datetime
! pip install numpy

In [2]:
# import needed packages
import pandas as pd
import string
from datetime import datetime
import numpy as np

In [3]:
# load fish data
fish = pd.read_csv('../original_data/FISH.csv')
# fish dataframe
fish.head(3)

,FIELDID,Latitude,Longitude,Date and Time,Common Name,Quantity
0,Low4,42.286604,-83.475661,8/25/2015,Gizzard shad,1
1,Low9,42.283461,-83.505311,8/22/2016,Gizzard shad,9
2,Low11,42.297215,-83.525437,8/22/2016,Gizzard shad,11


In [4]:
# Capitalize common name words for easier merging
fish['Common Name'] = fish['Common Name'].apply(lambda x: string.capwords(x))
# Convert date to datetime object
fish['Date and Time'] = fish['Date and Time'].apply(lambda x: datetime.strptime(x,'%m/%d/%Y')) 

In [7]:
# load species data
categories = pd.read_csv('../modified_data/fish/P51_SPECIES.csv')
# add a count variable
categories['Value'] = 1
# species dataframe
categories.head(3)

,Common Name,Category,Value
0,Sea Lamprey,Intolerant,1
1,Silver Lamprey,Intolerant,1
2,Northern Brook Lamprey,Intolerant,1


In [8]:
# create pivot table for categories, by common name
# 1 indicates True, 0 indicates False
pivot_categories = pd.pivot_table(categories, index=['Common Name'], columns=['Category'], fill_value=0, values='Value')
pivot_categories.head()

Category,Darter,Insectivores,Intolerant,Lithophilic,Omnivores,Piscivores,Sucker,Sunfish,Tolerant
Common Name,,,,,,,,,
American Brook Lamprey,0,0,1,0,0,0,0,0,0
American Eel,0,0,0,0,0,1,0,0,0
Arctic Grayling,0,1,1,0,0,0,0,0,0
Atlantic Salmon,0,0,1,0,0,0,0,0,0
Banded Darter,1,1,1,1,0,0,0,0,0


In [9]:
# merge pivot table with fish dataframe
samples = fish.merge(pivot_categories, on=["Common Name"], how='left').fillna(0)
samples.sample(3)

,FIELDID,Latitude,Longitude,Date and Time,Common Name,Quantity,Darter,Insectivores,Intolerant,Lithophilic,Omnivores,Piscivores,Sucker,Sunfish,Tolerant
2178,Low6,42.282611,-83.407008,2016-05-10,Round Goby,28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491,Fowl2,42.282260,-83.505150,2016-08-22,Pumpkinseed,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5388,HamL,42.606682,-83.322362,2018-08-29,Iowa Darter,24,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# calculate quantity of fish per species per sample
samples['Number Omnivores'] = samples['Quantity'] * samples['Omnivores']
samples['Number Insectivores'] = samples['Quantity'] * samples['Insectivores']
samples['Number Piscivores'] = samples['Quantity'] * samples['Piscivores']
samples['Number Tolerant'] = samples['Quantity'] * samples['Tolerant']
samples['Number Lithophilic'] = samples['Quantity'] * samples['Lithophilic']
samples.head()

,FIELDID,Latitude,Longitude,Date and Time,Common Name,Quantity,Darter,Insectivores,Intolerant,Lithophilic,Omnivores,Piscivores,Sucker,Sunfish,Tolerant,Number Omnivores,Number Insectivores,Number Piscivores,Number Tolerant,Number Lithophilic
0,Low4,42.286604,-83.475661,2015-08-25,Gizzard Shad,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Low9,42.283461,-83.505311,2016-08-22,Gizzard Shad,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Low11,42.297215,-83.525437,2016-08-22,Gizzard Shad,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LR-6,42.285000,-83.383888,2013-09-06,Gizzard Shad,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Fowl2,42.282260,-83.505150,2014-08-22,Gizzard Shad,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# calculate statistics
samples_stats = samples.groupby(['FIELDID', 'Date and Time', 'Latitude', 'Longitude']).agg({
    'Quantity': 'sum', # add all quantities
    'Common Name': 'nunique', # unique number of species
    'Number Omnivores': 'sum',  # total number of omnivore
    'Number Insectivores': 'sum',
    'Number Piscivores': 'sum',
    'Number Tolerant': 'sum',
    'Number Lithophilic': 'sum',
    'Darter':'sum',# total number of darter species
    'Sunfish':'sum',
    'Sucker':'sum',
    'Intolerant': 'sum'
    
})

In [12]:
samples_stats.reset_index(inplace=True)
# statistics dataframe
samples_stats.sample(3)

,FIELDID,Date and Time,Latitude,Longitude,Quantity,Common Name,Number Omnivores,Number Insectivores,Number Piscivores,Number Tolerant,Number Lithophilic,Darter,Sunfish,Sucker,Intolerant
107,John9,2012-05-24,42.380660,-83.545192,153,13,59.0,71.0,3.0,89.0,11.0,10.0,15.0,5.0,9.0
376,Main20,2021-08-20,42.330060,-83.241500,130,10,33.0,97.0,0.0,41.0,8.0,1.0,3.0,1.0,0.0
393,Min4,2012-05-15,42.526938,-83.421444,429,11,233.0,182.0,0.0,318.0,154.0,10.0,6.0,13.0,0.0


In [13]:
# caculate percent 
samples_stats['Percent Omnivore'] = samples_stats['Number Omnivores'] / samples_stats['Quantity']
samples_stats['Percent Insectivores'] = samples_stats['Number Insectivores'] / samples_stats['Quantity']
samples_stats['Percent Piscivores'] = samples_stats['Number Piscivores'] / samples_stats['Quantity']
samples_stats['Percent Tolerant'] = samples_stats['Number Tolerant'] / samples_stats['Quantity']
samples_stats['Percent Lithophilic'] = samples_stats['Number Lithophilic'] / samples_stats['Quantity']

In [14]:
samples_stats.head()

,FIELDID,Date and Time,Latitude,Longitude,Quantity,Common Name,Number Omnivores,Number Insectivores,Number Piscivores,Number Tolerant,Number Lithophilic,Darter,Sunfish,Sucker,Intolerant,Percent Omnivore,Percent Insectivores,Percent Piscivores,Percent Tolerant,Percent Lithophilic
0,Bell1,2016-09-16,42.42924,-83.39666,233,10,110.0,120.0,1.0,140.0,128.0,8.0,5.0,17.0,0.0,0.472103,0.515021,0.004292,0.600858,0.549356
1,Bell4,2016-09-16,42.40327,-83.36237,721,10,636.0,85.0,0.0,699.0,45.0,6.0,10.0,6.0,0.0,0.882108,0.117892,0.000000,0.969487,0.062413
2,Bell4,2022-06-17,42.40327,-83.36237,117,8,68.0,49.0,0.0,109.0,6.0,1.0,1.0,1.0,0.0,0.581197,0.418803,0.000000,0.931624,0.051282
3,Bish2,2017-06-23,42.47131,-83.45151,185,8,72.0,113.0,0.0,164.0,16.0,1.0,3.0,0.0,0.0,0.389189,0.610811,0.000000,0.886486,0.086486
4,Bish2,2022-07-06,42.47131,-83.45151,83,7,75.0,4.0,4.0,78.0,10.0,1.0,2.0,1.0,0.0,0.903614,0.048193,0.048193,0.939759,0.120482


In [15]:
# drop disposable columns
samples_stats = samples_stats.drop(['Number Omnivores', 
                                    'Number Insectivores', 'Number Piscivores', 'Number Tolerant', 'Number Lithophilic'], axis=1)

In [16]:
samples_stats

,FIELDID,Date and Time,Latitude,Longitude,Quantity,Common Name,Darter,Sunfish,Sucker,Intolerant,Percent Omnivore,Percent Insectivores,Percent Piscivores,Percent Tolerant,Percent Lithophilic
0,Bell1,2016-09-16,42.429240,-83.396660,233,10,8.0,5.0,17.0,0.0,0.472103,0.515021,0.004292,0.600858,0.549356
1,Bell4,2016-09-16,42.403270,-83.362370,721,10,6.0,10.0,6.0,0.0,0.882108,0.117892,0.000000,0.969487,0.062413
2,Bell4,2022-06-17,42.403270,-83.362370,117,8,1.0,1.0,1.0,0.0,0.581197,0.418803,0.000000,0.931624,0.051282
3,Bish2,2017-06-23,42.471310,-83.451510,185,8,1.0,3.0,0.0,0.0,0.389189,0.610811,0.000000,0.886486,0.086486
4,Bish2,2022-07-06,42.471310,-83.451510,83,7,1.0,2.0,1.0,0.0,0.903614,0.048193,0.048193,0.939759,0.120482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Wall7,2017-06-23,42.509150,-83.495620,244,9,0.0,3.0,0.0,0.0,0.586066,0.413934,0.000000,0.598361,0.000000
452,WalnutL,2018-08-24,42.566600,-83.325630,920,10,2.0,5.0,0.0,3.0,0.830435,0.084783,0.028261,0.834783,0.000000
453,Will1,2015-10-16,42.330020,-83.464220,195,6,1.0,0.0,1.0,0.0,0.938462,0.056410,0.000000,0.969231,0.343590
454,Will1,2022-07-26,42.330020,-83.464220,239,8,1.0,2.0,1.0,0.0,0.656904,0.330544,0.004184,0.974895,0.330544


In [18]:
# save as .csv
samples_stats.to_csv('../modified_data/fish/fish_stats.csv')  